## Create login as sysadmin
The following snippet activates the mixed mode authentication and creates the sysadmin login

In [ ]:

USE [master]
GO
/* [Note: 2 indicates mixed mode authentication. 1 is for windows only authentication] */
EXEC xp_instance_regwrite N'HKEY_LOCAL_MACHINE', 
     N'Software\Microsoft\MSSQLServer\MSSQLServer',
     N'LoginMode', REG_DWORD, 2
GO
/* Restart SQL Server engine */
-- TODO: translate in PowerShell

In [ ]:
USE MRT_NEW
/* Creates new login */
CREATE LOGIN testsysadmin WITH PASSWORD = 'ABCDegf123';
GO
/* Assigns the sysadmin server role */
EXEC master..sp_addsrvrolemember @loginame = N'testsysadmin', @rolename = N'sysadmin'
GO

## Perform manual backup
https://docs.microsoft.com/it-it/sql/relational-databases/backup-restore/create-a-full-database-backup-sql-server?view=sql-server-ver15

This requires the roles _sysadmin_, _db_owner_ or _db_backupoperator_. 
The backup file is automatically named "[DATABASE_NAME]-[DBVER]_[yyyyMMdd_HHmm].bak".

In [8]:
USE MRT_NEW

/* Variables */
DECLARE @VERSION NVARCHAR(255); SELECT @VERSION = CONCAT(SUBSTRING(T05VALORE,1,1),SUBSTRING(T05VALORE,3,2)) FROM T05COMFLAGS WHERE T05TIPO = 'DBVER'
DECLARE @FILENAME NVARCHAR(255); SET @FILENAME = DB_NAME() + '-' + @VERSION +'_' + FORMAT(GETDATE(),'yyyyMMdd_hhmm') + '.bak'
DECLARE @FULLPATH NVARCHAR(255); SET @FULLPATH = "C:\MPW\" + @FILENAME

/* Backup database MRT */
BACKUP DATABASE MRT_NEW
TO DISK = @FULLPATH
   WITH FORMAT,
      MEDIANAME = 'SQLServerBackups',
      NAME = 'Full Backup of MRT';
GO


Processed 912 pages for database 'MRT_NEW', file 'MRT_NEW' on file 1.

Processed 2 pages for database 'MRT_NEW', file 'MRT_NEW_log' on file 1.

BACKUP DATABASE successfully processed 914 pages in 0.087 seconds (82.076 MB/sec).

Total execution time: 00:00:00.131

## Perform manual restore
https://docs.microsoft.com/it-it/sql/t-sql/statements/restore-statements-transact-sql?view=sql-server-ver15

This requires the roles _sysadmin_, _db_owner_ or _db_backupoperator_. 

In [ ]:
USE MRT_NEW

/* Variables */
DECLARE @FULLPATH NVARCHAR(255)
SET @FULLPATH = 'C:\MPW\MRT*.bak'
DECLARE @RESTOREDDBNAME NVARCHAR(60)
SET @RESTOREDDBNAME = 'MRT_RESTORED'

/* Instructions */
RESTORE DATABASE @RESTOREDDBNAME
FROM DISK = @FULLPATH
WITH  FILE = 1, 
NOUNLOAD,  
STATS = 5

GO

## Query di stato impianto
Alcune query per avere una panoramica dell'impianto

In [ ]:
USE MRT_NEW

-- Versione installata
SELECT T05VALORE AS [Versione installata] FROM T05COMFLAGS WHERE T05TIPO='DBVER'

-- Servizi btService con rispettiva GNetPath
SELECT T03CODICE AS Codice, T03DESCRIZIONE AS Descrizione, 
CASE T03CONFIGGN WHEN '' THEN 'KARM' ELSE T03CONFIGGN END AS [GNet Path] FROM T03COMSERVICES

-- Famiglie di firmware presenti sul campo
SELECT T22GNTYPE AS [Versione Firmware],COUNT(T22CODICE) AS [Terminali base attivi] FROM T22ACCTERMINALI WHERE T22KK='0' AND T22ABILITATO='1' GROUP BY T22GNTYPE

-- Terminali base attivi
SELECT T22CODICE AS Codice, T22DESCRIZIONE AS Descrizione, T22GNTYPE AS [Firmware], T22GNNUN AS RamoNodo, T22GNIP AS IndirizzoIP FROM T22ACCTERMINALI WHERE T22KK='0' AND T22ABILITATO='1'



## Query su HOTLINE
Inserire il cliente nella variabile @MioCliente ed eventualmente il tecnico nella variabile @Tecnico

In [ ]:
/* Query su HOTLINE */

USE ASSISTENZA
-- Variabile
DECLARE @MioCliente NVARCHAR(MAX) 
SET @MioCliente = '%NORD MILANO%'

-- Chiamate per cliente
SELECT 
    ORA AS [DATA ORA],
    CLIENTE AS CLIENTE,
    CONCAT(DESCR_TH,' ', note_hotline) AS DESCRIZIONE,
    DURATA_th AS DURATA,
    TECNICO_HOT AS TECNICO
FROM t_INTERVENTI
WHERE Cliente like @MioCliente
AND TECNICO_HOT like '%SANNA%'
AND DURATA_TH IS NOT NULL
ORDER BY DATA DESC